# 📊 Prévision de la production hydro-électrique

Dans cette section, nous allons nous concentrer sur **la centrale hydro-électrique située dans la région de Montpellier**.  
L’objectif est de prédire la **production quotidienne d’énergie** à partir de données environnementales et hydrologiques locales.

- `date_obs_elab` : date de l’observation (YYYY-MM-DD)  
- `prod_hydro` : production d’énergie (kW)  


## Import 

In [10]:
import pandas as pd
import matplotlib.pyplot as plt

## 1. Chargement des données

In [11]:
df = pd.read_csv("~/production_enr/data/prod_hydro.csv")
display(df.head(6))

,date_obs_elab,prod_hydro
0,2026-08-01,-200.0
1,2022-08-01,200.0
2,2022-09-01,200.0
3,2022-10-01,200.0
4,2022-11-01,200.0
5,2022-12-01,0.0


## 2. Dimensions et types de variables

In [12]:
df.dtypes

date_obs_elab     object
prod_hydro       float64
dtype: object

## 3. Valeurs manquantes

In [13]:
missing = df.isna().sum()

print(f"Missing values: {missing}")

Missing values: date_obs_elab    0
prod_hydro       0
dtype: int64


## 4. Statistiques descriptives

In [14]:
display(df.describe())

,prod_hydro
count,997.000000
mean,191.745588
std,72.255973
min,-200.000000
25%,200.000000
50%,200.000000
75%,200.000000
max,2000.000000


## 5. Valeurs aberrantes
- **d’une centrale hydro (200 kW nominaux)**

In [ ]:
df_clean = df[(df['prod_hydro'] <= 200)  & (df['prod_hydro'] >= 0)]
display(df_clean)
# j'ai supprimé la valeur abberante
display(df_clean.describe())

,date_obs_elab,prod_hydro
1,2022-08-01,200.0
2,2022-09-01,200.0
3,2022-10-01,200.0
4,2022-11-01,200.0
5,2022-12-01,0.0
...,...,...
992,2025-02-19,200.0
993,2025-02-20,200.0
994,2025-02-21,200.0
995,2025-02-22,0.0


,prod_hydro
count,997.000000
mean,191.745588
std,72.255973
min,-200.000000
25%,200.000000
50%,200.000000
75%,200.000000
max,2000.000000
